# Federated learning (Client 2)

In [ ]:
%run CLIENT2/client2.py

## Baseline (Client2)

In [ ]:
%run CLIENT2/baseline/client2.py

## Knowledge Distillation (Client2)

In [1]:
%run CLIENT2/KD_0/client2.py

2024-08-14 20:24:36.538124: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-14 20:24:36.538169: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO flwr 2024-08-14 20:24:40,557 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)


[2024-08-14 20:24:40,557][flwr][INFO] - Opened insecure gRPC connection (no certificates were passed)


DEBUG flwr 2024-08-14 20:24:40,559 | connection.py:42 | ChannelConnectivity.IDLE


[2024-08-14 20:24:40,559][flwr][DEBUG] - ChannelConnectivity.IDLE


DEBUG flwr 2024-08-14 20:24:40,560 | connection.py:42 | ChannelConnectivity.CONNECTING


...

[2024-08-14 20:24:40,563][flwr][DEBUG] - ChannelConnectivity.READY
client 2
client 3
*************Metrics******************
Healthy: 0.7724696355649823, Benign disease: 0.30338733427048786, Malignant disease: 0.692733017322849
Epoch [1/30] - train loss 0.8309 - train accuracy 0.6406 - train f1_micro 0.6406 - train f1_weighted 0.6299
model 2
*************Metrics******************
Healthy: 0.5568181815018078, Benign disease: 0.6494845354129026, Malignant disease: 0.5879120875890593
val loss 0.9035 - val accuracy 0.5890 - val f1_micro 0.5890 - val f1_weighted 0.6095
Local best ACC achieved...... 0.6095135101914124
*************Metrics******************
Healthy: 0.4089068825579833, Benign disease: 0.02209131074785106, Malignant disease: 0.5963665086416773
Epoch [2/30] - train loss 1.1688 - train accuracy 0.4009 - train f1_micro 0.4009 - train f1_weighted 0.3592
model 2
*************Metrics******************
Healthy: 0.9999999994318182, Benign disease: 0.0, Malignant disease: 0.0
val loss 1

*************Metrics******************
Healthy: 0.8445344128870823, Benign disease: 0.0706921943931234, Malignant disease: 0.7969984201582151
Epoch [18/30] - train loss 0.7565 - train accuracy 0.6604 - train f1_micro 0.6604 - train f1_weighted 0.6052
model 2
*************Metrics******************
Healthy: 0.7784090904668131, Benign disease: 0.10309278339887343, Malignant disease: 0.8571428566718996
val loss 0.7411 - val accuracy 0.6659 - val f1_micro 0.6659 - val f1_weighted 0.6223
*************Metrics******************
Healthy: 0.8591093116713273, Benign disease: 0.0868924889415475, Malignant disease: 0.7922590836656984
Epoch [19/30] - train loss 0.7523 - train accuracy 0.6676 - train f1_micro 0.6676 - train f1_weighted 0.6164
model 2
*************Metrics******************
Healthy: 0.9034090903957903, Benign disease: 0.030927835019662026, Malignant disease: 0.6813186809443305
val loss 0.8017 - val accuracy 0.6286 - val f1_micro 0.6286 - val f1_weighted 0.5610
*************Metrics*****

DEBUG flwr 2024-08-14 21:14:18,857 | connection.py:141 | gRPC channel closed


[2024-08-14 21:14:18,857][flwr][DEBUG] - gRPC channel closed


KeyboardInterrupt: 

In [ ]:
%run CLIENT2/KD_1/client2.py

## SKD (Client2)

In [ ]:
%run CLIENT2/SKD/client2.py

## Analyse baseline (Client2)

In [ ]:
%run CLIENT2/baseline_layers/client2.py

## LowSimilarity (Client2)

In [ ]:
%run CLIENT2/lowsimilarity/client2.py

## Ensemble (Client2)

In [ ]:
%run CLIENT2/ensemble/client2.py

## Ensemble dataset (Client2)

In [ ]:
%run CLIENT2/ensemble_dataset/client2.py

## t-SNE (Client2)

In [ ]:
%run CLIENT2/baseline/test_tSNE.py

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from glob import glob
import cv2
import numpy as np

class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_ids, img_dir, img_ext):
        self.img_ids = img_ids
        self.img_dir = img_dir
        self.img_ext = img_ext

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]                                                                                                 
        img = cv2.imdecode(np.fromfile(os.path.join(self.img_dir, img_id + '.jpg'), dtype=np.uint8), cv2.IMREAD_COLOR)
        
               
        img = img.astype('float32') #/ 255
        img = img.transpose(2, 0, 1)

        #mask = mask.transpose(2, 0, 1)

        return torch.from_numpy(img).to(torch.float32)

def getStat(train_data):
    '''
    Compute mean and variance for training data
    :param train_data: Self dataset / folder
    :return: (mean, std)
    '''
    print('Compute mean and variance for training data.')
    print(len(train_data))
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=1, shuffle=False, num_workers=0,
        pin_memory=True)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    for X  in train_loader:  #torch.Size([1, 3, 1038, 1388]) [1, 574, 765, 3]
        X = (X.detach().numpy())/255
        for d in range(3):
            mean[d] += X[:, d, :, :].mean()
            std[d] += X[:, d, :, :].std()
    mean.div_(len(train_data))
    std.div_(len(train_data))
    return list(mean.numpy()), list(std.numpy())
 
if __name__ == '__main__':
    img_ids = glob(os.path.join("isic2019_all", '*' + '.jpg'))
    img_ids = [os.path.splitext(os.path.basename(p))[0] for p in img_ids]

    sum_dataset = Dataset(
        img_ids=img_ids,
        img_dir=os.path.join("isic2019_all"),
        img_ext='.jpg')

    print(getStat(sum_dataset))

In [ ]:
if "fc" in "model.fc.bias":
    print("1")
else:
    print("2")